# NF2 original analytical field 
> calculation

In [ ]:
import os
import pyvista as pv 
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from scipy.integrate import solve_bvp
from astropy.nddata import block_replicate
from tqdm import tqdm
from tvtk.api import tvtk, write_data

In [ ]:
def get_analytic_b_field(n=1, m=1, l=0.3, psi=np.pi / 2, resolution=64, bounds=[-1, 1, -1, 1, 0, 2]):
    """
    Calculate the analytic NLFF field from Low & Lou (1989).

    :param n: variable see Low & Lou (1989), only works for n=1
    :param m: used for generating a proper initial condition.
    :param a2: eigenvalue
    :param l: depth below the photosphere
    :param psi: angle of the magnetic field relative to the dipol axis
    :param resolution: spatial resolution of the magnetic field in pixels
    :param bounds: dimensions of the volume (x_start, x_end, y_start, y_end, z_start, z_end)
    :return: magnetic field B (x, y, z, v)
    """
    sol_P, a2 = solve_P(n, m)

    resolution = [resolution] * 3 if not isinstance(resolution, list) else resolution
    coords = np.stack(np.meshgrid(np.linspace(bounds[0], bounds[1], resolution[1], dtype=np.float32),
                                  np.linspace(bounds[2], bounds[3], resolution[0], dtype=np.float32),
                                  np.linspace(bounds[4], bounds[5], resolution[2], dtype=np.float32)), -1).transpose(
        [1, 0, 2, 3])

    x, y, z = coords[..., 0], coords[..., 1], coords[..., 2]
    X = x * np.cos(psi) - (z + l) * np.sin(psi)
    Y = y
    Z = x * np.sin(psi) + (z + l) * np.cos(psi)

    # to spherical coordinates
    xy = X ** 2 + Y ** 2
    r = np.sqrt(xy + Z ** 2)
    theta = np.arctan2(np.sqrt(xy), Z)
    phi = np.arctan2(Y, X)

    mu = np.cos(theta)

    P, dP_dmu = sol_P(mu)
    A = P / r ** n
    dA_dtheta = -np.sin(theta) / (r ** n) * dP_dmu
    dA_dr = P * (-n * r ** (-n - 1))
    Q = np.sqrt(a2) * A * np.abs(A) ** (1 / n)

    Br = (r ** 2 * np.sin(theta)) ** -1 * dA_dtheta
    Btheta = - (r * np.sin(theta)) ** -1 * dA_dr
    Bphi = (r * np.sin(theta)) ** -1 * Q

    BX = Br * np.sin(theta) * np.cos(phi) + Btheta * np.cos(theta) * np.cos(phi) - Bphi * np.sin(phi)
    BY = Br * np.sin(theta) * np.sin(phi) + Btheta * np.cos(theta) * np.sin(phi) + Bphi * np.cos(phi)
    BZ = Br * np.cos(theta) - Btheta * np.sin(theta)

    Bx = BX * np.cos(psi) + BZ * np.sin(psi)
    By = BY
    Bz = - BX * np.sin(psi) + BZ * np.cos(psi)

    b_field = np.real(np.stack([Bx, By, Bz], -1))
    return b_field


def solve_P(n, m):
    """
    Solve the differential equation from Low & Lou (1989).

    :param n: variable (only n=1)
    :param v0: start condition for dP/dmu
    :param P0: boundary condition for P(-1) and P(1)
    :return: interpolated functions for P and dP/dmu
    """

    def f(x, y, p):
        a2 = p[0]
        d2P_dmu2 = -(n * (n + 1) * y[0] + a2 * (1 + n) / n * y[0] ** (1 + 2 / n)) / (1 - x ** 2 + 1e-6)
        return [y[1], d2P_dmu2]

    def f_boundary(Pa, Pb, p):
        return np.array([Pa[0] - 0, Pb[0] - 0, Pa[1] - 10])

    mu = np.linspace(-1, 1, num=256)

    if m % 2 == 0:
        init = np.cos(mu * (m + 1) * np.pi / 2)
    else:
        init = np.sin(mu * (m + 1) * np.pi / 2)

    dinit = 10 * np.ones_like(init)  #
    initial = np.stack([init, dinit])

    @np.vectorize
    def shooting(a2_init):
        eval = solve_bvp(f, f_boundary, x=mu, y=initial, p=[a2_init], verbose=0, tol=1e-6)
        if eval.success == False:
            return None
        return eval

    # use shooting to find eigenvalues
    evals = shooting(np.linspace(0, 10, 100, dtype=np.float32))
    evals = [e for e in evals if e is not None]

    eigenvalues = np.array([e.p for e in evals])
    eigenvalues = sorted(set(np.round(eigenvalues, 4).reshape((-1,))))

    # get final solution
    eval = shooting([eigenvalues[-1]])[0]

    return eval.sol, eval.p[0]

In [ ]:
class PotentialModel(nn.Module):

    def __init__(self, b_n, r_p):
        super().__init__()
        self.register_buffer('b_n', b_n)
        self.register_buffer('r_p', r_p)
        c = np.zeros((1, 3))
        c[:, 2] = (1 / np.sqrt(2 * np.pi))
        c = torch.tensor(c, dtype=torch.float32, )
        self.register_buffer('c', c)

    def forward(self, coord):
        v1 = self.b_n[:, None]
        v2 = 2 * np.pi * ((-self.r_p[:, None] + coord[None, :] + self.c[None]) ** 2).sum(-1) ** 0.5
        potential = torch.sum(v1 / v2, dim=0)
        return potential


def get_potential(b_n, height, batch_size=2048, strides=(1, 1, 1), progress=True):
    cube_shape = (*b_n.shape, height)
    strides = (strides, strides, strides) if isinstance(strides, int) else strides
    b_n = b_n.reshape((-1,)).astype(np.float32)
    coords = np.stack(np.mgrid[:cube_shape[0]:strides[0], :cube_shape[1]:strides[1], :cube_shape[2]:strides[2]], -1).reshape((-1, 3))
    r_p = np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], :1], -1).reshape((-1, 3))

    # torch code
    # r = (x * y, 3); coords = (x*y*z, 3), c = (1, 3)
    # --> (x * y, x * y * z, 3) --> (x * y, x * y * z) --> (x * y * z)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    with torch.no_grad():
        b_n = torch.tensor(b_n, dtype=torch.float32, )
        r_p = torch.tensor(r_p, dtype=torch.float32, )
        model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

        coords = torch.tensor(coords, dtype=torch.float32)
        potential = []
        loader = DataLoader(TensorDataset(coords), batch_size=batch_size, num_workers=8)
        it = tqdm(loader, desc='Potential Field') if progress else loader
        for coord, in it:
            coord = coord.to(device)
            p_batch = model(coord)
            potential += [p_batch]

    potential = torch.cat(potential).view(cube_shape).cpu().numpy()
    if strides != (1, 1, 1):
        potential = block_replicate(potential, strides, conserve_sum=False)
    return potential


def get_potential_boundary(b_n, height, batch_size=2048):
    assert not np.any(np.isnan(b_n)), 'Invalid data value'

    cube_shape = (*b_n.shape, height)

    b_n = b_n.reshape((-1)).astype(np.float32)
    coords = [np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], cube_shape[2] - 2:cube_shape[2] + 1], -1),
              np.stack(np.mgrid[:cube_shape[0], -1:2, :cube_shape[2]], -1),
              np.stack(np.mgrid[:cube_shape[0], cube_shape[1] - 2:cube_shape[1] + 1, :cube_shape[2]], -1),
              np.stack(np.mgrid[-1:2, :cube_shape[1], :cube_shape[2]], -1),
              np.stack(np.mgrid[cube_shape[0] - 2:cube_shape[0] + 1, :cube_shape[1], :cube_shape[2]], -1), ]
    coords_shape = [c.shape[:-1] for c in coords]
    flat_coords = np.concatenate([c.reshape(((-1, 3))) for c in coords])

    r_p = np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], :1], -1).reshape((-1, 3))

    # torch code
    # r = (x * y, 3); coords = (x*y*z, 3), c = (1, 3)
    # --> (x * y, x * y * z, 3) --> (x * y, x * y * z) --> (x * y * z)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    with torch.no_grad():
        b_n = torch.tensor(b_n, dtype=torch.float32, )
        r_p = torch.tensor(r_p, dtype=torch.float32, )
        model = nn.DataParallel(PotentialModel(b_n, r_p, )).to(device)

        flat_coords = torch.tensor(flat_coords, dtype=torch.float32, )

        potential = []
        for coord, in tqdm(DataLoader(TensorDataset(flat_coords), batch_size=batch_size, num_workers=2),
                           desc='Potential Boundary'):
            coord = coord.to(device)
            p_batch = model(coord)
            potential += [p_batch.cpu()]

    potential = torch.cat(potential).numpy()
    idx = 0
    fields = []
    for s in coords_shape:
        p = potential[idx:idx + np.prod(s)].reshape(s)
        b = - 1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)
        fields += [b]
        idx += np.prod(s)

    fields = [fields[0][:, :, 1].reshape((-1, 3)),
              fields[1][:, 1, :].reshape((-1, 3)), fields[2][:, 1, :].reshape((-1, 3)),
              fields[3][1, :, :].reshape((-1, 3)), fields[4][1, :, :].reshape((-1, 3))]
    coords = [coords[0][:, :, 1].reshape((-1, 3)),
              coords[1][:, 1, :].reshape((-1, 3)), coords[2][:, 1, :].reshape((-1, 3)),
              coords[3][1, :, :].reshape((-1, 3)), coords[4][1, :, :].reshape((-1, 3))]
    return np.concatenate(coords), np.concatenate(fields)

In [ ]:
def pv_plot(B, vtk_path='./evaluation.vtk', points=((7, 64, 8), (7, 64, 8)), overwrite=False):

    if not os.path.exists(vtk_path) or overwrite:
        dim = B.shape[:-1]
        pts = np.stack(np.mgrid[0:dim[0], 0:dim[1], 0:dim[2]], -1).astype(np.float32)
        pts = pts.transpose(2, 1, 0, 3)
        pts = pts.reshape((-1, 3))
        vectors = B.transpose(2, 1, 0, 3)
        vectors = vectors.reshape((-1, 3))
        sg = tvtk.StructuredGrid(dimensions=dim, points=pts)
        sg.point_data.vectors = vectors
        sg.point_data.vectors.name = 'B'
        write_data(sg, str(vtk_path))

    mesh = pv.read(vtk_path)
    xindmax, yindmax, zindmax = mesh.dimensions
    xcenter, ycenter, zcenter = mesh.center

    p = pv.Plotter(off_screen=True)
    p.add_mesh(mesh.outline())

    sargs_B = dict(
        title='Bz [G]',
        title_font_size=15,
        height=0.25,
        width=0.05,
        vertical=True,
        position_x = 0.05,
        position_y = 0.05,
    )
    dargs_B = dict(
        scalars='B', 
        component=2, 
        clim=(-150, 150), 
        scalar_bar_args=sargs_B, 
        show_scalar_bar=False, 
        lighting=False
    )
    p.add_mesh(mesh.extract_subset((0, xindmax, 0, yindmax, 0, 0)), 
            cmap='gray', **dargs_B)

    def draw_streamlines(pts):
        stream, src = mesh.streamlines(
            return_source=True,
            start_position = pts,
            integration_direction='both',
            max_time=1000,
        )
        # print(pts)
        key = pts[0]*pts[1] + (pts[0]//pts[1]) + (pts[0] - pts[1])
        # print(key)
        np.random.seed(key)
        colors = np.random.rand(3)
        # if pts[0] == 16 and pts[1] == 48:
        #     colors = 'white'
        # print(colors)
        p.add_mesh(stream.tube(radius=0.2), lighting=False, color=colors)
        p.add_mesh(src, point_size=7, color=colors)

    xrange = points[0]
    yrange = points[1]
    for i in np.arange(*xrange):
        for j in np.arange(*yrange):
            try: 
                draw_streamlines((i, j, 0))
            except:
                pass
                # print(i, j)

    p.camera_position = 'xy'
    p.show_bounds()
    # p.add_title(title)

    return p

In [ ]:
b = get_analytic_b_field()
potential = get_potential(b[:, :, 0, 2], b.shape[2], batch_size=1000)
b_potential = - 1 * np.stack(np.gradient(potential, axis=[0, 1, 2], edge_order=2), axis=-1)

Potential Field: 100%|██████████████████████| 263/263 [00:00<00:00, 322.13it/s]


In [ ]:
def save_vtk(B, vtk_path):
    dim = B.shape[:-1]
    pts = np.stack(np.mgrid[0:dim[0], 0:dim[1], 0:dim[2]], -1).astype(np.float32)
    pts = pts.transpose(2, 1, 0, 3)
    pts = pts.reshape((-1, 3))
    vectors = B.transpose(2, 1, 0, 3)
    vectors = vectors.reshape((-1, 3))
    sg = tvtk.StructuredGrid(dimensions=dim, points=pts)
    sg.point_data.scalars = np.linalg.norm(vectors, axis=-1)
    sg.point_data.scalars.name = 'mag'
    sg.point_data.vectors = vectors
    sg.point_data.vectors.name = 'B'
    write_data(sg, str(vtk_path))

In [ ]:
save_vtk(B=b, vtk_path='b_nf2.vtk')
save_vtk(B=b_potential, vtk_path='bp_nf2.vtk')

## Visualization

In [ ]:
import pyvista as pv

In [ ]:
pv.start_xvfb()
pv.global_theme.trame.server_proxy_enabled = True

In [ ]:
from zpinn.pinn_nf2_visualization import draw_grid

In [ ]:
import pickle

### Original NLFFF

In [ ]:
b_mesh = pv.read('b_nf2.vtk')
b_mesh

StructuredGrid (0x7f153163f760)
  N Cells:      250047
  N Points:     262144
  X Bounds:     0.000e+00, 6.300e+01
  Y Bounds:     0.000e+00, 6.300e+01
  Z Bounds:     0.000e+00, 6.300e+01
  Dimensions:   64, 64, 64
  N Arrays:     2

In [ ]:
b_nf22 = draw_grid(b_mesh)
# b_nf22.plt_Bz()

In [ ]:
b_nf22.pv_streamline()

Widget(value="<iframe src='/proxy/34011/index.html?ui=P_0x7f1531950f10_0&reconnect=auto' style='width: 99%; he…

### My NLFFF

In [ ]:
with open("b.pickle","rb") as f:
    b = pickle.load(f)

In [ ]:
bb = draw_grid(b.grid)
# bb.plt_Bz()

In [ ]:
bb.pv_streamline()

Widget(value="<iframe src='/proxy/34011/index.html?ui=P_0x7f15240b5e40_1&reconnect=auto' style='width: 99%; he…

### Original Potential Field

In [ ]:
bp_mesh = pv.read('bp_nf2.vtk')
bp_mesh

StructuredGrid (0x7f152326d6c0)
  N Cells:      250047
  N Points:     262144
  X Bounds:     0.000e+00, 6.300e+01
  Y Bounds:     0.000e+00, 6.300e+01
  Z Bounds:     0.000e+00, 6.300e+01
  Dimensions:   64, 64, 64
  N Arrays:     2

In [ ]:
bp_nf22 = draw_grid(bp_mesh)
# bp_nf22.plt_Bz()

In [ ]:
bp_nf22.pv_streamline()

Widget(value="<iframe src='/proxy/34011/index.html?ui=P_0x7f15233040d0_2&reconnect=auto' style='width: 99%; he…

### My Potential

In [ ]:
from zpinn.pinn_nf2_potential import get_potential_field

In [ ]:
with open("bp.pickle","rb") as f:
    bp = pickle.load(f)

In [ ]:
bpbp = draw_grid(bp.grid)
# bpbp.plt_Bz()

In [ ]:
bpbp.pv_streamline()

Widget(value="<iframe src='/proxy/34011/index.html?ui=P_0x7f1504138a30_3&reconnect=auto' style='width: 99%; he…